In [1]:
# Part 1: Populating NCBI_RefSeq Table

In [1]:
import requests
import re

r = requests.get('https://ftp.ncbi.nlm.nih.gov/refseq/release/release-notes/')   
line = r.text.split('\n')[10]
fileName = line[line.find('>')+1:]
index = fileName.find('<')

fileName = fileName[:index]
date=re.findall('\d\d\d\d-\d\d-\d\d',line)[0]
num = re.findall('\d\d\d', fileName)[0]
fileName = "https://ftp.ncbi.nlm.nih.gov/refseq/release/release-notes/" + fileName

In [3]:
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    user="mubil",
    passwd="***Mubil520^^^",
    database='SCRISPRdb'
)

mycursor = mydb.cursor()

query = 'INSERT INTO NCBI_RefSeq(release_num, release_date, release_notes, comments) VALUES ('+ num + ', \''+ date + '\', \''+ fileName + '\', \'\');'
mycursor.execute(query)

mydb.commit()
mycursor.close()

True

In [4]:
# Part 2: Populating Complete_Genomes - Archaea

In [2]:
import pandas as pd
import mysql.connector
import math
import re

mydb = mysql.connector.connect(
    host="localhost",
    user="mubil",
    passwd="***Mubil520^^^",
    database='SCRISPRdb'
)

mycursor = mydb.cursor()


df = pd.read_csv("assembly_summary_archaea.txt", delimiter='\t', skiprows=1) 

df.drop(df[df['assembly_level'] != 'Complete Genome'].index, inplace = True) 
df.drop(df[df['version_status'] != 'latest'].index, inplace = True) 



for index, row in df.iterrows():
    ftp = re.findall('GCF_.*$', row['ftp_path'])[0] + '_genomic.crisp'
    
    file = open('/home/liangl5/CRISPR/crf/CRISPRs_archaea/' + ftp, 'r')
    refSeqAccession = next(file).split(' ')[2]
    genomeLength = int(next(file).split(': ')[1])
    file.close();
    
    
    check=" "+num+',\''+row['# assembly_accession']+'\',\''+row['# assembly_accession']+'\',\''+refSeqAccession+'\',\''+row['refseq_category']+'\','+str(row['taxid'])+','+str(row['species_taxid'])
    
    isPlaceHolder = str(row['infraspecific_name'])
    iPlaceHolder = str(row['isolate'])
    oPlaceHolder = row['organism_name']
    
    if (isPlaceHolder == 'nan'):
        isPlaceHolder = ''
    if (iPlaceHolder == 'nan'):
        iPlaceHolder = ''
    oPlaceHolder = oPlaceHolder.replace('\'', '\\\'')
    
    check = check+',\''+oPlaceHolder+'\',\''+isPlaceHolder+'\',\''+iPlaceHolder+'\','+str(genomeLength)+',\'archaea\',\''+row['ftp_path'] + '\''
    query = 'INSERT INTO Complete_Genome(release_num,RefSeqAssembly_Accession,GenBankAssembly_Accession,RefSeq_Accession,RefSeqCategory,taxId,species_taxId,organism_name,infraspecific_name,isolate,genome_length,domain,fasta_file_name)'
    query = query + ' VALUES (' + check + ');'
    mycursor.execute(query)
    mydb.commit()
mycursor.close()

True

In [16]:
# Part 2.5: Populating Complete_Genomes - Bacteria

In [4]:
import pandas as pd
import mysql.connector
import math
import re

mydb = mysql.connector.connect(
    host="localhost",
    user="mubil",
    passwd="***Mubil520^^^",
    database='SCRISPRdb'
)

mycursor = mydb.cursor()


df = pd.read_csv("assembly_summary_bacteria.txt", delimiter='\t', skiprows=1) 

df.drop(df[df['assembly_level'] != 'Complete Genome'].index, inplace = True) 
df.drop(df[df['version_status'] != 'latest'].index, inplace = True) 



for index, row in df.iterrows():
    ftp = re.findall('GCF_.*$', row['ftp_path'])[0] + '_genomic.crisp'
    
    file = open('/home/liangl5/CRISPR/crf/CRISPRs_bacteria/' + ftp, 'r')
    refSeqAccession = next(file).split(' ')[2]
    genomeLength = int(next(file).split(': ')[1])
    file.close();
    
    check=" "+num+',\''+row['# assembly_accession']+'\',\''+row['# assembly_accession']+'\',\''+refSeqAccession+'\',\''+row['refseq_category']+'\','+str(row['taxid'])+','+str(row['species_taxid'])    
    
    isPlaceHolder = str(row['infraspecific_name'])
    iPlaceHolder = str(row['isolate'])
    oPlaceHolder = row['organism_name']
    
    if (isPlaceHolder == 'nan'):
        isPlaceHolder = ''
    if (iPlaceHolder == 'nan'):
        iPlaceHolder = ''
    oPlaceHolder = oPlaceHolder.replace('\'', '\\\'')
    isPlaceHolder = isPlaceHolder.replace('\'', '\\\'')
    check = check+',\''+oPlaceHolder+'\',\''+isPlaceHolder+'\',\''+iPlaceHolder+'\','+str(genomeLength)+',\'bacteria\',\''+row['ftp_path'] + '\''
    
   
    query = 'INSERT INTO Complete_Genome(release_num,RefSeqAssembly_Accession,GenBankAssembly_Accession,RefSeq_Accession,RefSeqCategory,taxId,species_taxId,organism_name,infraspecific_name,isolate,genome_length,domain,fasta_file_name)'
    query = query + ' VALUES (' + check + ');'
    #print(query)
    mycursor.execute(query)
    mydb.commit()
mycursor.close()

True

In [3]:
# Part 3: Populating CRISPRs - Archaea

In [8]:
import os
import mysql.connector
import re

mydb = mysql.connector.connect(
    host="localhost",
    user="mubil",
    passwd="***Mubil520^^^",
    database='SCRISPRdb'
)

mycursor = mydb.cursor()


rangeEnd = 0
rangeStart = 0
repeatPositions = ""
spacerPositions = ""
prevRepeatLength = 0

repeatCount = 0
for filename in os.listdir('/home/liangl5/CRISPR/crf/CRISPRs_archaea'):
    file1 = open('/home/liangl5/CRISPR/crf/CRISPRs_archaea/' + filename, 'r') 
    fileLines = file1.readlines() 
    
    # This finds the accession id so we can see which genome it is
    tmpMatch = re.findall('\.\d', filename)[0]
    tmpIndex = filename.find(tmpMatch)
    accession = filename[:tmpIndex+2]
    
    for line in fileLines:
        if (line.startswith('Repeats')):
            # This means we reached the end of a crispr and need to add into table
            
            # This finds the genome_id
            mycursor.execute('SELECT genome_id FROM Complete_Genome WHERE RefSeqAssembly_Accession = \'' + accession + '\';')
            records = mycursor.fetchall()
            genomeId = records[0][0]
            
            query = 'INSERT INTO CRISPRs(genome_id, range_begin, range_end, repeat_positions, spacer_positions, repeat_count, spacer_count) VALUES '
            query = query + '(' + str(genomeId) + ',' + str(rangeStart) + ',' + str(rangeEnd) + ',\'' + repeatPositions[1:] + '\',\''
            query = query + spacerPositions[1:] + '\',' + str(repeatCount) + ',' + str(repeatCount-1) + ');'
            #print(query)
            mycursor.execute(query)
            mydb.commit()
            
            # reset global variable
            repeatCount = 0
            repeatPositions = ""
            spacerPositions = ""
        
        # Means new crispr
        if (line.startswith('CRISPR')):
        
            index = line.find(':')
            rangeArray = line[index+2:-1].split(' - ')
            
            rangeStart = rangeArray[0]
            rangeEnd = rangeArray[1]

        # Means an entry into a particular crispr
        if (line[0].isdigit()):
            tmpArray = line.split('\t')
            spacerLength = 0
            
            if (tmpArray[-1][0] == '['):
                genomeLengths = tmpArray[-1][2:-3].split(', ')
                spacerLength = int(genomeLengths[1])
                prevRepeatLength = int(genomeLengths[0])
            repeatStart = int(tmpArray[0])
              
                
            shortrepeat = str(repeatStart) + '-' + str(repeatStart + prevRepeatLength - 1)
            if (spacerLength != 0):
                spacerPositions = spacerPositions + ":" + str(repeatStart+prevRepeatLength) + '-' + str(repeatStart+prevRepeatLength+spacerLength-1)
            repeatPositions = repeatPositions + ":" + shortrepeat
            repeatCount+=1
    
mycursor.close()

True

In [ ]:
# Part 3: Populating CRISPRs - Bacteria

In [9]:
import os
import mysql.connector
import re

mydb = mysql.connector.connect(
    host="localhost",
    user="mubil",
    passwd="***Mubil520^^^",
    database='SCRISPRdb'
)

mycursor = mydb.cursor()


rangeEnd = 0
rangeStart = 0
repeatPositions = ""
spacerPositions = ""
prevRepeatLength = 0

repeatCount = 0
for filename in os.listdir('/home/liangl5/CRISPR/crf/CRISPRs_bacteria'):
    file1 = open('/home/liangl5/CRISPR/crf/CRISPRs_bacteria/' + filename, 'r') 
    fileLines = file1.readlines() 
    
    # This finds the accession id so we can see which genome it is
    tmpMatch = re.findall('\.\d', filename)[0]
    tmpIndex = filename.find(tmpMatch)
    accession = filename[:tmpIndex+2]
    
    for line in fileLines:
        if (line.startswith('Repeats')):
            # This means we reached the end of a crispr and need to add into table
            
            # This finds the genome_id
            mycursor.execute('SELECT genome_id FROM Complete_Genome WHERE RefSeqAssembly_Accession = \'' + accession + '\';')
            records = mycursor.fetchall()
            genomeId = records[0][0]
            
            query = 'INSERT INTO CRISPRs(genome_id, range_begin, range_end, repeat_positions, spacer_positions, repeat_count, spacer_count) VALUES '
            query = query + '(' + str(genomeId) + ',' + str(rangeStart) + ',' + str(rangeEnd) + ',\'' + repeatPositions[1:] + '\',\''
            query = query + spacerPositions[1:] + '\',' + str(repeatCount) + ',' + str(repeatCount-1) + ');'
            #print(query)
            mycursor.execute(query)
            mydb.commit()
            
            # reset global variable
            repeatCount = 0
            repeatPositions = ""
            spacerPositions = ""
        
        # Means new crispr
        if (line.startswith('CRISPR')):
        
            index = line.find(':')
            rangeArray = line[index+2:-1].split(' - ')
            
            rangeStart = rangeArray[0]
            rangeEnd = rangeArray[1]

        # Means an entry into a particular crispr
        if (line[0].isdigit()):
            tmpArray = line.split('\t')
            spacerLength = 0
            
            if (tmpArray[-1][0] == '['):
                genomeLengths = tmpArray[-1][2:-3].split(', ')
                spacerLength = int(genomeLengths[1])
                prevRepeatLength = int(genomeLengths[0])
            repeatStart = int(tmpArray[0])
              
                
            shortrepeat = str(repeatStart) + '-' + str(repeatStart + prevRepeatLength - 1)
            if (spacerLength != 0):
                spacerPositions = spacerPositions + ":" + str(repeatStart+prevRepeatLength) + '-' + str(repeatStart+prevRepeatLength+spacerLength-1)
            repeatPositions = repeatPositions + ":" + shortrepeat
            repeatCount+=1
    
mycursor.close()

True